In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 32919014
paper_name = 'guan_zhang_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table S1 Normolized counts.xlsx', sheet_name='69samples normolized bigger tha', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3766 x 49


In [7]:
original_data.head()

,Unnamed: 0,DMSO_1,DMSO_2,DMSO_3,Pro_max_1,Pro_max_2,Pro_max_3,Pro_med_1,Pro_med_2,Pro_med_3,...,Flu_min_3,Epo_max_1,Epo_max_2,Epo_max_3,Epo_med_1,Epo_med_2,Epo_med_3,Epo_min_1,Epo_min_2,Epo_min_3
0,YAL001C,243,246,214,301,249,253,300,309,358,...,266,228,252,327,305,228,282,331,380,282
1,YAL003W,283,336,376,391,323,392,369,347,328,...,270,303,423,437,442,413,474,376,477,312
2,YAL033W,122,212,167,204,175,193,202,165,222,...,198,21,13,173,172,178,213,192,234,156
3,YAL038W,173,163,154,103,172,140,101,109,117,...,145,170,101,144,162,119,130,145,195,112
4,YAL043C,122,112,130,50,118,112,135,93,112,...,109,99,65,133,104,164,140,108,90,112


In [8]:
original_data['orf'] = original_data['Unnamed: 0'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, DMSO_1, DMSO_2, DMSO_3, Pro_max_1, Pro_max_2, Pro_max_3, Pro_med_1, Pro_med_2, Pro_med_3, Pro_min_1, Pro_min_2, Pro_min_3, Pen_max_1, Pen_max_2, Pen_max_3, Pen_med_1, Pen_med_2, Pen_med_3, Pen_min_1, Pen_min_2, Pen_min_3, Teb_max_1, Teb_max_2, Teb_max_3, Teb_med_1, Teb_med_2, Teb_med_3, Teb_min_1, Teb_min_2, Teb_min_3, Flu_max_1, Flu_max_2, Flu_max_3, Flu_med_1, Flu_med_2, Flu_med_3, Flu_min_1, Flu_min_2, Flu_min_3, Epo_max_1, Epo_max_2, Epo_max_3, Epo_med_1, Epo_med_2, Epo_med_3, Epo_min_1, Epo_min_2, Epo_min_3, orf]
Index: []

[0 rows x 50 columns]


In [16]:
original_data.drop(columns=['Unnamed: 0'], inplace=True)

In [12]:
original_data.set_index('orf', inplace=True)

In [17]:
cols = ['_'.join(c.split("_")[0:-1]) for c in original_data.columns]

In [23]:
original_data.columns = cols

In [24]:
original_data = original_data.T

In [25]:
original_data = original_data.groupby(original_data.index.values).mean().T

In [26]:
original_data.head()

,DMSO,Epo_max,Epo_med,Epo_min,Flu_max,Flu_med,Flu_min,Pen_max,Pen_med,Pen_min,Pro_max,Pro_med,Pro_min,Teb_max,Teb_med,Teb_min
orf,,,,,,,,,,,,,,,,
YAL001C,234.333333,261.000000,251.000000,271.666667,267.666667,322.333333,273.333333,248.666667,273.666667,224.666667,269.000000,271.666667,331.000000,292.333333,298.000000,287.000000
YAL003W,331.666667,296.666667,385.666667,389.000000,368.666667,348.000000,463.666667,342.000000,422.333333,333.000000,387.666667,443.000000,388.333333,361.333333,361.666667,364.000000
YAL033W,167.000000,176.333333,93.666667,168.666667,190.666667,196.333333,108.000000,209.666667,184.333333,129.333333,69.000000,187.666667,194.000000,127.666667,119.666667,122.333333
YAL038W,163.333333,146.666667,167.666667,131.000000,138.333333,109.000000,153.333333,141.666667,132.000000,151.666667,138.333333,137.000000,150.666667,152.666667,146.666667,158.333333
YAL043C,121.333333,124.000000,109.000000,126.666667,93.333333,113.333333,118.000000,100.666667,87.333333,136.000000,99.000000,136.000000,103.333333,119.666667,120.666667,119.666667


In [27]:
original_data.shape

(3766, 16)

In [31]:
orfs_essential = original_data.index.values[is_essential(original_data.index.values)]
orfs_nonessential = original_data.index.values[~is_essential(original_data.index.values)]

Loading SGD features from 2017-04-03.
Loading SGD features from 2017-04-03.


In [32]:
original_data1 = original_data.loc[orfs_nonessential,:].copy()
original_data2 = original_data.loc[orfs_essential,:].copy()

In [39]:
for c in original_data1.columns:
    if not 'DMSO' in c:
        original_data1[c] = original_data1[c] / original_data1['DMSO']

In [40]:
for c in original_data2.columns:
    if not 'DMSO' in c:
        original_data2[c] = original_data2[c] / original_data2['DMSO']

In [54]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_noness', rsuffix='_ess')

In [55]:
original_data.shape

(3802, 32)

In [56]:
original_data.head()

,DMSO_noness,Epo_max_noness,Epo_med_noness,Epo_min_noness,Flu_max_noness,Flu_med_noness,Flu_min_noness,Pen_max_noness,Pen_med_noness,Pen_min_noness,...,Flu_min_ess,Pen_max_ess,Pen_med_ess,Pen_min_ess,Pro_max_ess,Pro_med_ess,Pro_min_ess,Teb_max_ess,Teb_med_ess,Teb_min_ess
orf,,,,,,,,,,,,,,,,,,,,,
YAL001C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.166430,1.061166,1.167852,0.958748,1.147937,1.159317,1.412518,1.247511,1.271693,1.224751
YAL003W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.397990,1.031156,1.273367,1.004020,1.168844,1.335678,1.170854,1.089447,1.090452,1.097487
YAL024C,137.666667,0.651332,0.871671,0.743341,0.878935,0.924939,0.736077,0.825666,1.094431,0.876513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YAL033W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.646707,1.255489,1.103792,0.774451,0.413174,1.123752,1.161677,0.764471,0.716567,0.732535
YAL034C,37.333333,0.348214,0.946429,0.223214,0.455357,0.669643,0.758929,0.267857,0.562500,0.571429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Prepare the final dataset

In [57]:
data = original_data.copy()

In [58]:
original_data.columns

Index(['DMSO_noness', 'Epo_max_noness', 'Epo_med_noness', 'Epo_min_noness',
       'Flu_max_noness', 'Flu_med_noness', 'Flu_min_noness', 'Pen_max_noness',
       'Pen_med_noness', 'Pen_min_noness', 'Pro_max_noness', 'Pro_med_noness',
       'Pro_min_noness', 'Teb_max_noness', 'Teb_med_noness', 'Teb_min_noness',
       'DMSO_ess', 'Epo_max_ess', 'Epo_med_ess', 'Epo_min_ess', 'Flu_max_ess',
       'Flu_med_ess', 'Flu_min_ess', 'Pen_max_ess', 'Pen_med_ess',
       'Pen_min_ess', 'Pro_max_ess', 'Pro_med_ess', 'Pro_min_ess',
       'Teb_max_ess', 'Teb_med_ess', 'Teb_min_ess'],
      dtype='object')

In [59]:
dataset_ids_noness = [21931, 21886, 21921, 21922, 21885, 21927, 21928, 21883, 21923, 21924, 21882, 21925, 21926, 21884, 21929, 21930]
dataset_ids_ess = [21934, 21937, 21936, 21935, 21940, 21938, 21939, 21943, 21942, 21941, 21946, 21945, 21944, 21949, 21948, 21947]
dataset_ids = dataset_ids_noness + dataset_ids_ess
datasets = datasets.reindex(index=dataset_ids)

In [60]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [61]:
data.head()

dataset_id,21931,21886,21921,21922,21885,21927,21928,21883,21923,21924,...,21939,21943,21942,21941,21946,21945,21944,21949,21948,21947
data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,,,
YAL001C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.166430,1.061166,1.167852,0.958748,1.147937,1.159317,1.412518,1.247511,1.271693,1.224751
YAL003W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.397990,1.031156,1.273367,1.004020,1.168844,1.335678,1.170854,1.089447,1.090452,1.097487
YAL024C,137.666667,0.651332,0.871671,0.743341,0.878935,0.924939,0.736077,0.825666,1.094431,0.876513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YAL033W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.646707,1.255489,1.103792,0.774451,0.413174,1.123752,1.161677,0.764471,0.716567,0.732535
YAL034C,37.333333,0.348214,0.946429,0.223214,0.455357,0.669643,0.758929,0.267857,0.562500,0.571429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Subset to the genes currently in SGD

In [62]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 16


In [63]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21931,21886,21921,21922,21885,21927,21928,21883,21923,21924,...,21939,21943,21942,21941,21946,21945,21944,21949,21948,21947
,data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1,YAL001C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.166430,1.061166,1.167852,0.958748,1.147937,1.159317,1.412518,1.247511,1.271693,1.224751
3,YAL003W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.397990,1.031156,1.273367,1.004020,1.168844,1.335678,1.170854,1.089447,1.090452,1.097487
22,YAL024C,137.666667,0.651332,0.871671,0.743341,0.878935,0.924939,0.736077,0.825666,1.094431,0.876513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,YAL033W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.646707,1.255489,1.103792,0.774451,0.413174,1.123752,1.161677,0.764471,0.716567,0.732535
1861,YAL034C,37.333333,0.348214,0.946429,0.223214,0.455357,0.669643,0.758929,0.267857,0.562500,0.571429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
data.shape

(3786, 32)

# Normalize

In [65]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [66]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [67]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21931,21886,21921,21922,21885,21927,21928,21883,21923,21924,...,21939,21943,21942,21941,21946,21945,21944,21949,21948,21947
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1,YAL001C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.621946,0.370456,0.633847,-0.177045,0.482763,1.023089,1.837461,1.288742,1.197796,1.297662
3,YAL003W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.490162,0.252824,1.033666,0.035634,0.562960,1.778367,0.787306,0.547990,0.450134,0.767301
22,YAL024C,137.666667,0.651332,0.871671,0.743341,0.878935,0.924939,0.736077,0.825666,1.094431,0.876513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,YAL033W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.326714,1.132137,0.391111,-1.042838,-2.335720,0.870780,0.747425,-0.974986,-1.092234,-0.753608
36,YAL038W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.231625,-0.389254,-0.729094,-0.318810,-0.671839,-0.349653,-0.292145,-0.177250,-0.343947,0.233456


# Print out

In [68]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [69]:
from IO.save_data_to_db3 import *

In [70]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/32 [00:00<?, ?it/s]

Deleting all datasets for PMID 32919014...
Inserting the new data...


100%|██████████| 32/32 [01:53<00:00,  3.56s/it]

Updating the data_modified_on field...
